In [1]:
from pytheus import help_functions as hf, theseus as th, fancy_classes as fc, graphplot as gp
from neuralnet import prep_data, load_model
from datagen import generatorGraphFidelity
import matplotlib.pyplot as plt
import torch
import pandas as pd
import numpy as np
import re
from scipy import stats
import os
import argparse
import yaml
from yaml import Loader
import random


# NEEDS TO BE TESTED * 

# Loads up the training and test data. This assumes that the dataset corresponding to the trained neural network
# has been prepared using dataprep. 

def prep_data(cnfg, isTest):
    if(isTest):
        df = pd.read_csv(f"{cnfg['dataTest']}.csv", names=['weights','res'],delimiter=";", nrows=1200)
    else:
        df = pd.read_csv(f"{cnfg['dataTrain']}.csv", names=['weights', 'res'], delimiter=";", nrows=1200)
    try:
        weights = np.array([eval(graph) for graph in df['weights']])
    except:
        weights = np.array(
            [eval(re.sub(r"  *", ',', graph.replace('\n', '').replace('[ ', '['))) for graph in df['weights']])
    res = df['res'].to_numpy()
    return weights,res
        
    
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"

shift =1 

# Load up yaml files
print(shift)
stream = open(f"configs/analyze{shift}.yaml", 'r')
cnfg = yaml.load(stream, Loader=Loader)

weights_train, result_train = prep_data(cnfg,False)
weights_test, result_test = prep_data(cnfg,True)

nn_type = cnfg['nnType']
print(nn_type)
model = cnfg['model']
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("Device:", device)

mod = load_model(f'models/{model}.pt', 'cpu', 24, 1, nn_type)
DIM = [2] * 4
kets = hf.makeState('0000+1111')
state = fc.State(kets, normalize=True)
input_graph, ket_amplitudes, output_fidelity = generatorGraphFidelity(DIM, state, short_output=False)


ModuleNotFoundError: No module named 'neuralnet'

# Compute NN Prediction on the Model Output (Ideal GHZ State)

In [ ]:
for edge in input_graph.edges:
    input_graph[edge] = 0
    if len(DIM) == 6:
        if edge in [(0, 1, 0, 0), (2, 3, 0, 0), (4, 5, 0, 0), (1, 2, 1, 1), (3, 4, 1, 1), (0, 5, 1, 1)]:
            input_graph[edge] = 1

    else:
        if len(DIM)==4:
            if edge in [(0, 1, 0, 0), (2, 3, 0, 0), (1, 2, 1, 1), (0, 3, 1, 1)]:
                input_graph[edge] = 1

input_graph.getState()
input_graph.state.normalize()

 # predict fidelity of baseline (which ideally should be 1)
input = torch.tensor(input_graph.weights, dtype=torch.float).to(device)


# Compute NN Predictions on Training Data

In [ ]:
x_train = []
y_train = []

nnTitle = cnfg['name']

for ii in range(1000):
    input = torch.tensor(weights_train[ii], dtype=torch.float).to(device)
    x_train.append(float(mod(input)))
    y_train.append(result_train[ii])

# We compute the r^2 coefficient of a hypothetical linear fit between model predictions x and ground truths y 
 
slope_train, intercept_train, r_value_train, p_value_train, std_err_train = stats.linregress(x_train, y_train)

print("Results on training data")
print(f"Slope:{slope_train}, intercept:{intercept_train}, r_value:{r_value_train}")



# Compute NN Predictions on the test/validation data

In [ ]:
x_test = []
y_test = []

for ii in range(1000):
    input = torch.tensor(weights_test[ii], dtype=torch.float).to(device)
    x_test.append(float(mod(input)))
    y_test.append(result_test[ii])

slope_test, intercept_test, r_value_test, p_value_test, std_err_test = stats.linregress(x_test, y_test)

print("Results on test data")
print(f"Slope:{slope_test}, intercept:{intercept_test}, r_value:{r_value_test}")


# Now put it all together in a nice plot!

In [ ]:
import seaborn as sns


xLine = np.linspace(0,1)

fig, axes = plt.subplots(1, 2)

#Plot training results in one subplot

sns.lineplot(x=xLine*slope_train, y=xLine, ax=axes[0])
sns.scatterplot(x=x_train, y=y_train, ax=axes[0])
sns.scatterplot(x=float(mod(input)), y=1, color='green', ax=axes[0])
axes[0].set_title(f'True Fidelity vs. Predicted Fidelity -- {nnTitle} -- Training Data \n r^2 value:{r_value_train**2}')
axes[0].set_xlabel('Predicted Value')
axes[0].set_ylabel('True Value')

#plot test results in another subplot

sns.lineplot(x=xLine*slope_test, y=xLine, ax=axes[1], color='red')
sns.scatterplot(x=x_test, y=y_test, ax=axes[1], color='red')
sns.scatterplot(x=float(mod(input)), y=1, color='green', ax=axes[1])
axes[1].set_title(f'True Fidelity vs. Predicted Fidelity -- {nnTitle} -- Training Data \n r^2 value:{r_value_test**2}')
axes[1].set_xlabel('Predicted Value')
axes[1].set_ylabel('True Value')
